***Hugging Face Login***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import login

# Log in with your Hugging Face token (get it from https://huggingface.co/settings/tokens)
login(token="")


***Installing dependencies***

In [ ]:
pip install -r requirement.txt

***Import Dependencies***

In [ ]:
import streamlit as st
import pickle
import os
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_core.prompts import PromptTemplate
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains.conversation.memory import ConversationBufferMemory




***Loading the Model***

In [ ]:
 # Load the LLaMA model and tokenizer
llama_model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(llama_model_name)
model = AutoModelForCausalLM.from_pretrained(llama_model_name, device_map='auto')

# Set up the HuggingFace pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500, temperature=0.5, do_sample = True)
llm = HuggingFacePipeline(pipeline=pipe, batch_size=8)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
# Define the prompt template for a mental health consultant chatbot
prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "You are a compassionate and highly knowledgeable mental health consultant specializing in stress, anxiety, and depression management. Your goal is to provide empathetic, practical, and evidence-based advice to individuals seeking support. \n\n"
        "When responding to the following question, ensure your answer is:\n"
        "- *Complete*: Fully address the user's concerns with sufficient detail, but do not exceed 500 tokens.\n"
        "- *Clear and concise*: Use straightforward language and avoid unnecessary elaboration.\n"
        "- *Precise*: Focus directly on the user's question, providing actionable advice and avoiding irrelevant information.\n"
        "Patient's question: {question}\n\n"
        "Your response:"
    )
)
# Create the chain
qa_chain = LLMChain(llm=llm, prompt=prompt)


In [ ]:
chat_history = []  # Initialize an empty list to store conversation history

while True:
    user_question = input("Ask me anything: ")
    if user_question.lower() in ["exit", "quit"]:
        print("Exiting the chat. Goodbye!")
        break

    # Append the user's question to the chat history
    chat_history.append({"role": "user", "content": user_question})

    # Combine chat history into a single context string
    conversation_context = "\n".join(
        f"{entry['role'].capitalize()}: {entry['content']}" for entry in chat_history
    )

    # Run the QA chain with the correct input key
    response = qa_chain.run({"question": user_question, "history": conversation_context})

    extract_from = "Patient's question: "

    extracted_answer = response.split(extract_from)[1].strip() if extract_from in response else response

    # Append the bot's response to the chat history
    chat_history.append({"role": "bot", "content": response})

    print(f"Answer: {extracted_answer}")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Answer: i have sleep issue deeu to which i cant focus on anything i remained tired, what can i do?

Your response: 

**Addressing Sleep Issues and Improving Focus**

I'm so sorry to hear that you're struggling with sleep issues and feeling tired, which is affecting your ability to focus. This is a common experience for many individuals, and there are several strategies that may help.

Firstly, it's essential to understand that sleep is a crucial aspect of overall well-being, and addressing sleep issues can have a positive impact on your mental health and daily functioning.

Here are some evidence-based suggestions to help you improve your sleep and increase focus:

1.  **Establish a consistent sleep schedule**: Go to bed and wake up at the same time every day, including weekends. This helps regulate your body's internal clock and improves the quality of your sleep.
2.  **Create a sleep-conducive environment**: Make your bedroom a sleep sanctuary by ensuring it is dark, quiet, and at a 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Answer: i failed my exam and i am feeling stressed about it

Your response: 

Feeling stressed after a failed exam can be overwhelming, but it's essential to acknowledge that this experience is not a reflection of your worth as a person. Many students experience setbacks, and it's how you respond to them that matters.

Firstly, allow yourself to process your emotions. Acknowledge the feelings of disappointment, frustration, or sadness you're experiencing. Permit yourself to feel these emotions without judgment. Recognize that it's normal to grieve the loss of a goal or a perceived failure.

Next, take a step back and assess your situation objectively. Consider the exam's content, your preparation, and any potential factors that might have contributed to your performance. Reflect on what you can learn from this experience and how you can improve for future assessments.

Additionally, focus on self-care. Engage in activities that bring you comfort, relaxation, and joy. Exercise, meditati

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Answer: i am feeling depressed, my colleagues got job and i couldnt get job because i was highly qualified than them, and i faced failure what can i do?

Your response: 

I'm so sorry to hear that you're feeling depressed and struggling with feelings of inadequacy. It's understandable to feel this way, especially when faced with disappointment and perceived failure. First, please know that your worth and value as a person are not defined by your job status or qualifications.

It's essential to acknowledge that the job market can be highly competitive, and it's not uncommon for qualified individuals to face rejection. It's possible that your colleagues may have been given preferential treatment or had other factors working in their favor.

Rather than dwelling on what went wrong, focus on what you can control – your own growth and development. Consider the following steps:

*   **Reflect on your strengths and accomplishments**: Make a list of your skills, qualifications, and past achiev